The overall idea is that we will implement linear and tree based models first and try to ansamble them. Further will implement recurrent NN and compare the two models.
Data normalisation required for linear or NN models will be done with the help of sklearn pipeline which will include both motel and data transofrmation steps. 

In [1]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

from src.ToyModel import ToyModel
from src.utilities import run_cv
from src.FeatureGenerator import FeatureGenerator

# CV data initiation

In [2]:
# initiating data generators
feat_generator = FeatureGenerator()

# initiating cv splitter
tscv = TimeSeriesSplit(test_size = 2, max_train_size=11)

# generating all of the data which we will iterate over during CV
features_df = feat_generator.generate_features()

# creating col lists for training
cols_di={
    'index': feat_generator.index_cols,
    'target': feat_generator.target_col,
    'feats': feat_generator.shifted_cols + feat_generator.roll_cols
}


# Baseline

In [3]:
# initiating toy model
toy_model = ToyModel()

# iterating over CV folds
cv_res = run_cv(df=features_df, months_cv_split=tscv, model=toy_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23], size: 731,610
  Test months: [24 25],   size: 133,020
  NRMSE:  1.0
  RMSE :  2.7

Fold 1:
  Train months: [15 16 17 18 19 20 21 22 23 24 25], size: 731,610
  Test months: [26 27],   size: 133,020
  NRMSE:  1.0
  RMSE :  1.7

Fold 2:
  Train months: [17 18 19 20 21 22 23 24 25 26 27], size: 731,610
  Test months: [28 29],   size: 133,020
  NRMSE:  1.0
  RMSE :  1.9

Fold 3:
  Train months: [19 20 21 22 23 24 25 26 27 28 29], size: 731,610
  Test months: [30 31],   size: 133,020
  NRMSE:  1.0
  RMSE :  1.4

Fold 4:
  Train months: [21 22 23 24 25 26 27 28 29 30 31], size: 731,610
  Test months: [32 33],   size: 133,020
  NRMSE:  1.0
  RMSE :  1.3


------------------------------
RMSE mean: 1.8
NRMSE mean: 1.0


# Linear model

In [3]:
# initiating lin model
lin_model =  Pipeline([
    ("scaler", StandardScaler()),
    ("lin_model", LinearRegression()) #ElasticNet() Lasso(alpha=.005) , 
])

# iterating over CV folds
lin_cv_res = run_cv(df=features_df, months_cv_split=tscv, model=lin_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23], size: 731,610
  Test months: [24 25],   size: 133,020
  NRMSE:  0.7
  RMSE :  1.9

Fold 1:
  Train months: [15 16 17 18 19 20 21 22 23 24 25], size: 731,610
  Test months: [26 27],   size: 133,020
  NRMSE:  1.0
  RMSE :  1.7

Fold 2:
  Train months: [17 18 19 20 21 22 23 24 25 26 27], size: 731,610
  Test months: [28 29],   size: 133,020
  NRMSE:  0.72
  RMSE :  1.3

Fold 3:
  Train months: [19 20 21 22 23 24 25 26 27 28 29], size: 731,610
  Test months: [30 31],   size: 133,020
  NRMSE:  0.64
  RMSE :  0.88

Fold 4:
  Train months: [21 22 23 24 25 26 27 28 29 30 31], size: 731,610
  Test months: [32 33],   size: 133,020
  NRMSE:  0.81
  RMSE :  1.0


------------------------------
RMSE mean: 1.4
NRMSE mean: 0.78


# Tree based

In [4]:
# tree_model = RandomForestRegressor(n_estimators=300)
tree_model = XGBRegressor()

tree_cv_res = run_cv(df=features_df, months_cv_split=tscv, model=tree_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23], size: 731,610
  Test months: [24 25],   size: 133,020
  NRMSE:  0.85
  RMSE :  2.3

Fold 1:
  Train months: [15 16 17 18 19 20 21 22 23 24 25], size: 731,610
  Test months: [26 27],   size: 133,020
  NRMSE:  1.1
  RMSE :  1.8

Fold 2:
  Train months: [17 18 19 20 21 22 23 24 25 26 27], size: 731,610
  Test months: [28 29],   size: 133,020
  NRMSE:  0.71
  RMSE :  1.3

Fold 3:
  Train months: [19 20 21 22 23 24 25 26 27 28 29], size: 731,610
  Test months: [30 31],   size: 133,020
  NRMSE:  0.75
  RMSE :  1.0

Fold 4:
  Train months: [21 22 23 24 25 26 27 28 29 30 31], size: 731,610
  Test months: [32 33],   size: 133,020
  NRMSE:  0.65
  RMSE :  0.83


------------------------------
RMSE mean: 1.5
NRMSE mean: 0.8


# Recurrent NN